In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# inspired by https://stackoverflow.com/questions/7391945/how-do-i-read-image-data-from-a-url-in-python
from PIL import Image
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os
from google.colab import drive
from tensorflow.keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator


drive.mount('/content/drive')
# Set the random seed
np.random.seed(1337)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd drive/MyDrive/Year_5/ML-Artwork/data/

/content/drive/MyDrive/Year_5/ML-Artwork/data


In [ ]:
(img_x,img_y)=(300,300)
image_size=(img_x,img_y)
train_ds = keras.preprocessing.image_dataset_from_directory(
  "paintings_450",
  label_mode = "categorical",
  validation_split=0.2,
  subset="training",
  seed=1337,
  image_size=image_size,
  batch_size=36)

val_ds = keras.preprocessing.image_dataset_from_directory(
  "paintings_450",
  label_mode = "categorical",
  validation_split=0.2,
  subset="validation",
  seed=1337,
  image_size=image_size,
  batch_size=36)

Found 4411 files belonging to 10 classes.
Using 3529 files for training.
Found 4411 files belonging to 10 classes.
Using 882 files for validation.


In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255.,rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2, shear_range=0.2,
                                   zoom_range=0.2, horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale=1./255.)

In [ ]:
train_ds = train_datagen.flow_from_directory("paintings_450", batch_size=20,
                                             class_mode='binary',
                                             target_size=(img_x,img_y))

Found 4411 images belonging to 10 classes.


In [ ]:
# we apply random horizontal flipping or small random rotations to augment the dataset since we only have 450 images.
# 
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.15),
    ]
)

In [ ]:
# Apply `data_augmentation` to the training images.
# train_ds = train_ds.map(
#     lambda img, label: (data_augmentation(img), label),
#     num_parallel_calls=tf.data.AUTOTUNE,
# )
# Prefetching samples in GPU memory helps maximize GPU utilization.
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.prefetch(tf.data.AUTOTUNE)

In [ ]:
# help from https://www.analyticsvidhya.com/blog/2020/08/top-4-pre-trained-models-for-image-classification-with-python-code/#:~:text=Pre-Trained%20Models%20for%20Image%20Classification%201%201.%20Very,first%20model%20coming%20from%20the%20ResNet%20family.%20

base_model = VGG16(input_shape=(img_x,img_y,3),
                   include_top=False, weights='imagenet')
for layer in base_model.layers:
  layer.trainable = False

In [ ]:
x = layers.Flatten()(base_model.output)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(base_model.input, x)
model.compile(optimizer = tf.keras.optimizers.RMSprop(lr = 0.001),
              loss = 'binary_crossentropy', metrics = ['acc'])

In [ ]:
train_ds = train_ds.repeat()
val_ds = val_ds.repeat()

In [ ]:
model.fit(train_ds, steps_per_epoch=30,
          epochs=10)

Epoch 1/10
30/30 [==============================] - 534s 18s/step - loss: -8619.5664 - acc: 0.0900
Epoch 2/10
30/30 [==============================] - 531s 18s/step - loss: -32805.3398 - acc: 0.0883
Epoch 3/10
30/30 [==============================] - 509s 17s/step - loss: -61504.1406 - acc: 0.0846
Epoch 4/10
30/30 [==============================] - 524s 17s/step - loss: -95204.6484 - acc: 0.1067
Epoch 5/10
30/30 [==============================] - 517s 17s/step - loss: -144039.8906 - acc: 0.0800
Epoch 6/10
30/30 [==============================] - 520s 17s/step - loss: -194628.5938 - acc: 0.0967
Epoch 7/10
30/30 [==============================] - 522s 17s/step - loss: -256592.9844 - acc: 0.1017
Epoch 8/10
30/30 [==============================] - 518s 17s/step - loss: -334113.3750 - acc: 0.1017
Epoch 9/10
30/30 [==============================] - 529s 18s/step - loss: -409531.0000 - acc: 0.0983
Epoch 10/10
30/30 [==============================] - 524s 17s/step - loss: -520364.0938 - acc: 0